In [1]:
from geopandas import GeoDataFrame as df
import math
from shapely.geometry import LineString,Point
import json
%matplotlib inline
from geopandas import GeoSeries as ds
import pandas as pd

#from shape generate lines

In [2]:
shapes=df.from_csv('shapes.txt',index_col=None)
shapes['shape_seq']=shapes['shape_pt_sequence'].apply(lambda x: math.floor(x/10000))
shapes['shape_seq_id']=shapes[['shape_id','shape_seq']].apply(lambda x:str(x[0])+'-'+str(int(x[1])),axis=1)

In [3]:
distances=ds.from_csv('mta_shapes_meters_2014.csv',header=None)
speeds=ds.from_csv('speed.txt',sep='\t',header=None)
trips=df.from_csv('trips.txt')

In [4]:
line_dict={}
for shape_id in set(shapes.shape_seq_id):
    try:
        line=LineString(zip(shapes[shapes.shape_seq_id==shape_id]['shape_pt_lon'].values.tolist(),shapes[shapes.shape_seq_id==shape_id]['shape_pt_lat'].values.tolist()))
        speed=speeds[shape_id]
        distance=distances[shape_id]
        shape_num=shape_id.split('-')[0]
        route_name=list(set(trips[trips.shape_id==shape_num]['trip_headsign']))[0]
        route=list(set(trips[trips.shape_id==shape_num].index))[0]
        line_dict[shape_id]={}
        line_dict[shape_id]['speed']=speed
        line_dict[shape_id]['distance']=distance
        line_dict[shape_id]['geometry']=line
        line_dict[shape_id]['route_id']=route
        line_dict[shape_id]['route_name']=route_name
    except:
        pass

In [19]:
json_dump=df(line_dict).transpose()
json_dump.reset_index(inplace=True)
json_dump['shape_id']=json_dump['index'].apply(lambda x:x.split('-')[0])
json_dump['shape_seq']=json_dump['index'].apply(lambda x:x.split('-')[1])
json_dump.drop('index', axis=1, inplace=True)
json_dump=json_dump.drop_duplicates()

In [20]:
with open('brooklyn_0804_speed_distance.geojson','w') as f:
    f.write(json_dump.to_json())

#dump to redis

In [21]:
import redis
pool = redis.ConnectionPool(host='busvis.cloudapp.net', port=6379, db=10)
r = redis.Redis(connection_pool=pool)

In [24]:
for i in set(json_dump.route_id):
    r.hset('speed_distance',i,json_dump[json_dump.route_id==i].to_json())

#test

In [25]:
eval(r.hget('speed_distance','B38'))

{'features': [{'geometry': {'coordinates': [[-73.9046156752, 40.7009886473],
     [-73.9047270226, 40.7010540649],
     [-73.9054657362, 40.7014910911],
     [-73.9057853675, 40.7016804673]],
    'type': 'LineString'},
   'id': '2102',
   'properties': {'distance': '125.2',
    'route_id': 'B38',
    'route_name': 'DNTWN BKLYN TILLARY ST',
    'shape_id': 'B380099',
    'shape_seq': '1',
    'speed': 1.6702832192225066},
   'type': 'Feature'},
  {'geometry': {'coordinates': [[-73.918344817, 40.7051786536],
     [-73.9192284361, 40.7043094624],
     [-73.9194100752, 40.7041248674]],
    'type': 'LineString'},
   'id': '2103',
   'properties': {'distance': '147.64',
    'route_id': 'B38',
    'route_name': 'DNTWN BKLYN TILLARY ST',
    'shape_id': 'B380099',
    'shape_seq': '10',
    'speed': 2.410508020880758},
   'type': 'Feature'},
  {'geometry': {'coordinates': [[-73.9194100752, 40.7041248674],
     [-73.9208281236, 40.7026856755],
     [-73.9210783623, 40.7024305846]],
    'type': 

In [11]:
test=df.from_file('brooklyn_0804_speed_distance.geojson')



KeyError: False

In [23]:
test[(test['shape_id']=='B150225')]

,distance,geometry,route_id,route_name,shape_id,shape_seq,speed
545,190.57,"LINESTRING (-73.9414350182 40.699909328, -73.9...",B15,JFK AIRPORT,B150225,1,2.236341
546,164.44,"LINESTRING (-73.93859463140001 40.6857635298, ...",B15,JFK AIRPORT,B150225,10,4.269325
547,115.57,"LINESTRING (-73.93830392640001 40.6842993157, ...",B15,JFK AIRPORT,B150225,11,3.002433
548,167.06,"LINESTRING (-73.93810044439999 40.6832701933, ...",B15,JFK AIRPORT,B150225,12,4.216438
549,214.04,"LINESTRING (-73.9378054563 40.6817826447, -73....",B15,JFK AIRPORT,B150225,13,3.506045
550,154.09,"LINESTRING (-73.9374257867 40.6798770208, -73....",B15,JFK AIRPORT,B150225,14,2.541850
551,207.05,"LINESTRING (-73.9384783253 40.6793401627, -73....",B15,JFK AIRPORT,B150225,15,4.297168
552,172.19,"LINESTRING (-73.9386520103 40.6774804, -73.938...",B15,JFK AIRPORT,B150225,16,4.353811
553,185.07,"LINESTRING (-73.9383247155 40.6762461767, -73....",B15,JFK AIRPORT,B150225,17,4.596731
554,282.2,"LINESTRING (-73.93614118239999 40.6761280583, ...",B15,JFK AIRPORT,B150225,18,5.334074
